##Requirements
Before running this notebook make sure you have the following:
- An Azure ML workspace
- A compute instance

# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice
from azureml.data.dataset_type_definitions import PromoteHeadersBehavior
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model


import requests
import json
import pandas as pd
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#Retrieve current workspace
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Tuscan_Wine_Forecast'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



Workspace name: quick-starts-ws-162315
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-162315


## Compute Cluster
If compute cluster already exists, it will just identify it and end.


In [3]:
#Create Cluster if does not already exist

cluster_name = "myCluster"
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster already created")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",min_nodes=1, max_nodes=6)
    cluster = ComputeTarget.create(ws,cluster_name, compute_config) #creates the actual cluster

cluster.wait_for_completion(show_output=True)  #Allows to continue on other threads while cluster is being created


#SOURCE / HELP: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py

Cluster already created
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset
A Dataset is created from the csv file in the folder "data".

In [5]:
# Create a dataset from delimited files with header

#Retrieves default Datastore and display info
datastore = ws.get_default_datastore() 
print(datastore)

#Google Drive path to dl csv
url_data = "https://docs.google.com/spreadsheets/d/1X9M3eNuBDv0ZKsOdidkdaBx9W1NubNDz3kxXuULsXmo/export?format=csv"

#Loads into Dataframe
df = pd.read_csv(url_data)

#Save into Datastore AND Register as dataset
dataset = Dataset.Tabular.register_pandas_dataframe(df,datastore,'Dataset_Wine',description="A Dataset composed of Tuscan wines with ratings, vintage, grapes and corresponding season avg weather")

#dataset = Dataset.Tabular.from_delimited_files(path=url_data,validate=True, include_path=False,header=PromoteHeadersBehavior.ONLY_FIRST_FILE_HAS_HEADERS, support_multi_line=False)

datasetWineTuscan = dataset.register(workspace=ws, name='Dataset_Wine', description="A Dataset composed of Tuscan wines with ratings, vintage, grapes and corresponding season avg weather")


#Display top 10 lines
dataset.take(10).to_pandas_dataframe()


#----Previous exploration to upload file from within Azure

directory = os.getcwd()
print(directory)
#datastore.upload(src_dir='./data',
#                  target_path='datasets/',
#                  overwrite=True)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-75e762bd-4cf3-4ae8-8a22-58029b96effb",
  "account_name": "mlstrg162315",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/4d7082ae-de33-4ba9-846e-e26f55f7d063/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
/mnt/batch/tasks/shared/LS_root/mounts/clusters/amlcomputealeaume/code


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
#automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=45,
    task='classification',
    primary_metric='accuracy',
    training_data= dataset,
    validation_size = 0.20,
    label_column_name='points',
    compute_target = cluster,
    n_cross_validations=5
    )


#metric = normalized_root_mean_squared_error => recommended for review score prediction // for regression scenario

#AUC_weighted for classification

In [7]:
# TODO: Submit your experiment
runAutoML = experiment.submit(automl_config, show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Tuscan_Wine_Forecast,AutoML_cdbf3c17-d97a-49fd-9a95-f324124ae0fa,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(runAutoML).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
#WARNING:root:The model you attempted to retrieve requires 'azureml-train-automl-runtime' to be installed at '==1.34.1'. 
#Please install 'azureml-train-automl-runtime==1.34.1' (e.g. `pip install azureml-train-automl-runtime==1.34.1`) and then rerun the previous command.

import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'azureml-train-automl-runtime==1.34.1'])


In [10]:
best_run, fitted_model = runAutoML.get_output()
print(best_run)



print(fitted_model)

RunDetails(best_run).show()

Run(Experiment: Tuscan_Wine_Forecast,
Id: AutoML_cdbf3c17-d97a-49fd-9a95-f324124ae0fa_40,
Type: azureml.scriptrun,
Status: Completed)
None


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [11]:
#TODO: Save the best model

# Register run as Model

model_name = "AleaumeModelAutoML_2"
description = "Best AutoML Model"
model_path ="outputs/modelAutoML.pkl"

#model = best_run.register_model(model_name = model_name, description = description, model_path = model_path )

model = runAutoML.register_model(model_name = model_name, description = description)

#SOURCE / HELP: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

#Unfortunately so far unsuccessful to register the model via SDK and get a default generated swagger.json uri. Also,  MSFT support recommends to do so via AML studio...

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
#To create a local deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=9064)

#Create Environment

env = Environment(name="AzureML-AutoML")
#myenv=env.clone("myenv")
#myenv.python.conda_dependencies.add_pip_package("joblib==1.1.0")

#Uploaded manually entry scripts

my_inference_config = InferenceConfig(
    environment=env,
    source_directory= './',
    entry_script="./Users/odl_user_162315/score_dummy.py"
    #entry_script="./score.py"
)



# Deploy the service locally

service = model.deploy(ws, "local-service", [model], my_inference_config, deployment_config)
service.reload()
print(service.get_logs())

print(service.scoring_uri)

service.wait_for_deployment(show_output=True)


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_0549c39faee057510679b09a0f523620
 ---> e421954788f1
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 958ca73e481d
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImY1MDkxYzYwLTFjM2MtNDMwZi04ZDgxLWQ4MDJmNmJmMjQxNCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE2MjMxNSIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE2MjMxNSIsIndvcmtzcGFjZUlkIjoiNzVlNzYyYmQtNGNmMy00YWU4LThhMjItNTgwMjliOTZlZmZiIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in e91dafc53847
 ---> 7b900d7445f5
Step 4/5 : RUN mv '/var/azureml-app/tmpjlzqhjkb.py' /var/azureml-app/main.py
 ---> Running in 0f7c7f6673e2


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


 ---> d6ebf185c437
Step 5/5 : CMD ["runsvdir","/var/runit"]
 ---> Running in c2843970815a
 ---> 2412b72ff3ae
Successfully built 2412b72ff3ae
Successfully tagged local-service:latest
Container (name:stupefied_burnell, id:d5e1190b414639c9ce0e0b640da56aca4fd7cf3e855ab72f6e18ecd549817ae0) cannot be killed.
Container has been successfully cleaned up.
Image sha256:a250bee157ca1d7057c2b09ff6948ef7e4cf83b0d1072abb3ab6b5a7c1b9a09a successfully removed.
Starting Docker container...
Docker container running.
Container has been successfully cleaned up.
Starting Docker container...
Docker container running.
2021-10-28T22:10:58,895653031+00:00 - rsyslog/run 
2021-10-28T22:10:58,896997747+00:00 - iot-server/run 
2021-10-28T22:10:58,900033785+00:00 - gunicorn/run 
2021-10-28T22:10:58,900311689+00:00 - nginx/run 
Dynamic Python package installation is disabled.
Starting HTTP server
rsyslogd: /azureml-envs/azureml_38614f20eb97681ffe85531ba85caaed/lib/libuuid.so.1: no version information available (requi

In [21]:
#Call model to test 

#service.update(enable_app_insights=True)


uri = service.scoring_uri
requests.get("http://localhost:9064")
headers = {"Content-Type": "application/json"}
data = {
    "query": "This is a test query"
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

test is {'query': 'This is a test query'}


In [22]:
#Deploy to ACI

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=1, auth_enabled=True
)

service = model.deploy(
    ws,
    "mywebservice",
    [model],
    my_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-28 22:13:10+00:00 Creating Container Registry if not exists.
2021-10-28 22:13:10+00:00 Registering the environment.
2021-10-28 22:13:11+00:00 Use the existing image.
2021-10-28 22:13:11+00:00 Generating deployment configuration.
2021-10-28 22:13:12+00:00 Submitting deployment to compute.
2021-10-28 22:13:15+00:00 Checking the status of deployment mywebservice.

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

.
2021-10-28 22:24:23+00:00 Checking the status of inference endpoint mywebservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-10-28T22:23:57,872971100+00:00 - iot-server/run 
2021-10-28T22:23:57,874259500+00:00 - rsyslog/run 
2021-10-28T22:23:57,872971100+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-10-28T22:23:58,075937300+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_38614f20eb97681ffe85531ba85caaed/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-10-28T22:23:58,773652200+00:00 - iot-server/finish 1 0
2021-10-28T22:23:58,776822100+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (70)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 98
SPARK_HOME not set. Skipping PySpark Initialization.
Initializ

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
# Deleting the webservice

service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
